编程语言理论家把「一等对象」定义为满足以下条件的程序实体
- 运行时创建
- 能赋值给变量或数据结构中的元素
- 能作为参数传递给函数
- 能作为函数的返回结果

在 Python 中，整数、字符串、字典、函数都是一等对象。

In [1]:
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n * factorial(n-1)

factorial(42)

In [2]:
# 生成对象的帮助文本
factorial.__doc__

In [3]:
# function 类的实例
type(factorial)

In [4]:
# 将函数赋值给变量
fact = factorial
fact

In [5]:
fact(5)

In [6]:
# 返回可迭代对象，把第一个参数（函数）应用到第二个参数（可迭代对象）
map(factorial, range(11))

In [7]:
list(map(fact, range(11)))

函数式编程的特点之一是使用高阶函数

接收函数作为参数，或者把函数作为返回值的函数就是 **高阶函数（higher-order function）**

In [8]:
# 根据单词长度给列表排序
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']

# 任何单参数函数都能作为 key 参数的值
sorted(fruits, key=len)

In [9]:
# 根据反向拼写排序
def reverse(word):
    return word[::-1]

sorted(fruits, key=reverse)

在 Python 3 函数式编程范式中，最常见的高阶函数有 map、filter、reduce

Python 3 中，map 和 filter 返回生成器，因此它们的直接替代品是生成器表达式

sum 和 reduce 的通用思想是把某个操作连续应用到序列的元素上，累计之前的结果，把一系列值归约成一个值。

all 和 any 也是内置的归约函数
- all(iterable)
    - 所有元素都为真，返回 True
    - all([]) 返回 True
- any(iterable)
    - 任意元素为真，返回 True
    - any([]) 返回 Flase

In [10]:
# 0!~5! 的阶乘列表
list(map(fact, range(6)))

In [11]:
# 使用列表推导式
[fact(n) for n in range(6)]

In [12]:
# 0!~5! 的奇数阶乘列表
list(map(factorial, filter(lambda n: n % 2, range(6))))

In [13]:
# 使用列表推导式，避免 lambda 表达式
[factorial(n) for n in range(6) if n % 2]

In [14]:
from functools import reduce
from operator import add

# 0~99 累加和
reduce(add, range(100))

In [15]:
# 使用 sum 求和
sum(range(100))

lambda 关键字在 Python 表达式内创建匿名函数
lambda 函数定义体中不能赋值，也不能使用 while、try 等 Python 语句
在参数列表中最适合使用匿名函数
- 除了作为参数传给高阶函数之外，Python 很少使用匿名函数

lambda 句法只是语法糖：与 def 语句一样，lambda 表达式会创建函数对象。

匿名函数嵌套的层级太深不利于调试和处理错误，Python 中的异步变成结构更好，或许就是因为 lambda 表达式有局限。

In [16]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

Python 数据文档模型列出了 7 种可调用对象
1. 用户定义的函数
    - 使用 def 语句或 lambda 表达式创建
2. 内置函数
    - 使用 C 语言（CPython）事项的函数，如 len、time.strftime
3. 内置方法
    - 使用 C 语言实现的方法，如 dict.get
4. 方法
    - 在类的定义体中定义的函数
5. 类
    - 调用类相当于调用函数
    - 调用类时会运行类的 \_\_new__ 方法创建一个实例，然后运行 \_\_init__ 方法，初始化实例，最后把实例返回给调用方
6. 类的实例
    - 如果类定义了 \_\_call__ 方法，那么其实例可作为函数调用
7. 生成器函数
    - 使用 yield 关键字的函数或方法。调用生成器函数返回的是生成器对象
    - 生成器函数还可以作为协程

In [17]:
# 判断对象能否调用，最安全的方法是使用内置的 callable() 函数
[callable(obj) for obj in (abs, str, 13)]

In [18]:
import random

# 实例使用任何可迭代对象构建
# 在内部存储一个随机排序的列表
class BingoCage:

    def __init__(self, items):
        # 接受任何可迭代对象，在本地创建副本
        self._items = list(items)
        
        # 随机排列
        random.shuffle(self._items)
    
    # 取出一个元素
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    # 直接调用实例
    def __call__(self):
        return self.pick()

In [19]:
bingo = BingoCage(range(3))
bingo.pick()

In [20]:
# bingo.pick() 的快捷方式是 bingo()
bingo()

In [21]:
callable(bingo)

创建保有内部状态的函数
- 装饰器
    - 装饰器必须是函数，有时要在多次调用之间存储信息
- 闭包

函数使用 \_\_dict__ 属性存储赋予它的用户属性

In [22]:
# 查看函数对象属性
dir(factorial)

In [23]:
# 空的用户定义的类
class C:
    pass

# 创建实例
obj = C()

# 空函数
def func():
    pass

# 列出常规对象没有而函数有的属性
sorted(set(dir(func)) - set(dir(obj)))

名称|类型|说明
:-|:-|:-
\_\_annotations__|dict|参数和返回值的注解
\_\_call__|method-wrapper|实现 () 运算符，即可调用对象协议
\_\_closure__|tuple|函数闭包，即自由变量的绑定
\_\_code__|code|编译成字节码的函数元数据和函数定义体
\_\_defaults__|tuple|形式参数的默认值
\_\_get__|method-wrapepr|实现只读描述符协议
\_\_globals__|dict|函数在模块中的全局变量
\_\_kwdefaults__|dict|仅限关键字形式参数的默认值
\_\_name__|str|函数名称
\_\_qualname__|str|函数的限定名称

In [24]:
# 调用函数时使用 * 和 ** 展开可迭代对象，映射到单个参数
def tag(name, *content, cls=None, **attrs):
    """生成一个或多个 HTML 标签"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [25]:
# 单个位置参数
tag('br')

In [26]:
# 第一个参数后面的任意个参数会被 *content 捕获，存入元组
tag('p', 'hello', 'world')

In [27]:
# 没有明确指定名称的关键字参数会被 **attrs 捕获，存入字典
tag('p', 'hello', id=33)

In [28]:
# cls 参数只能作为关键字参数传入
tag('p', 'helo', 'world', cls='sidebar')

In [29]:
# 调用 tag 函数时，第一个位置参数也能作为关键字参数传入
tag(content='testing', name="img")

In [30]:
# 字典中所有的元素作为单个参数传入，同名键会绑定到对应的具名参数上，余下的被 **attrs 捕获
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
          'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

定义函数时若想指定仅限关键字参数，要把它们放到前面有 * 的参数后面；
如果不想支持不定的位置参数，但是想支持仅限关键字参数，在签名中放一个 *
仅限关键字参数不一定要有默认值

In [31]:
# b 必须传入实参
def f(a, *, b):
    return a, b

f(1, b=2)

函数对象的 \_\_defaults__ 属性保存着位置参数和关键字关键字参数的默认值，仅限关键字参数的默认值在 \_\_kwdefaults__ 属性中，而参数的名称在 \_\_code__ 属性中。

参数名称在 \_\_code__.co_varnames 中，里面包含了函数定义体中创建的局部变量
参数名称是前 N 个字符串，N 的值由 \_\_code__.co_argcount 确定。
- 不包含前缀为 * 和 \** 的变长参数
参数的默认值只能通过它们在 \_\_defaults__ 元组中的位置确定，因此要从后向前扫描才能把参数和默认值对应起来

In [32]:
def clip(text, max_len=80):
    """在 max_len 前面或后面的第一个空格处截断文本
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [33]:
clip.__defaults__

In [34]:
clip.__code__

In [35]:
clip.__code__.co_varnames

In [36]:
clip.__code__.co_argcount

In [37]:
# 提取函数的签名
from inspect import signature

sig = signature(clip)
sig

In [38]:
str(sig)

In [39]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

kind 属性的值是 \_ParameterKind 类中的 5 个值之一
- POSITIONAL_OR_KEYWORD
    - 可以通过位置参数和关键字参数传入
- VAR_POSITIONAL
    - 位置参数元组
- VAR_KEYWORD
    - 关键字参数字典
- KEYWORD_ONLY
    - 仅限关键字参数
- POSITIONAL_ONLY
    - 仅限位置参数

In [40]:
sig = signature(tag)
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
         'src': 'sunset.jpg', 'cls':'framed'}

# 把任意个参数绑定到签名中的形参
bound_args = sig.bind(**my_tag)
bound_args

In [41]:
for name, value in bound_args.arguments.items():
    print(name, '=', value)

In [42]:
del my_tag['name']
bound_args = sig.bind(**my_tag)

TypeError: missing a required argument: 'name'

函数声明中的各个参数可以在 `:` 之后添加注解表达式
如果参数有默认值，注解放在参数名和 = 号之间
如果想注解返回值，在 ) 和函数声明末尾的 : 之间添加 -> 和一个表达式（任意类型）
注解中最常用的类型是类（如 str、int）和字符串（如 'int > 0'）
注解只是元数据，可以供 IDE、框架、装饰器等工具使用
- 为 IDE 和 lint 程序等工具中的静态类型检查功能提供额外的类型信息

In [43]:
# 为函数声明中的参数和返回值附加元数据
def clip(text:str, max_len:'int > 0'=80) -> str:
    """在 max_len 前面或后面的第一个空格处截断文本
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [44]:
# 注解存储在函数的 __annotations__ 属性中
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

In [45]:
# 从函数签名中提取注解
sig = signature(clip)
sig.return_annotation

str

In [46]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


In [47]:
# 使用 reduce 函数和匿名函数
from functols import reduce

def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

ModuleNotFoundError: No module named 'functols'

In [48]:
# operator 模块为多个算术运算符提供了对应的函数
from operator import mul

def fact(n):
    return reduce(mul. range(1, n+1))

In [49]:
# operator 模块中还有一类函数，能替代从序列中取出元素或读取对象属性的 lambda 表达式
# itemgetter、attrgetter 会自行构建函数
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter

# 相当于 lambda fields: fields[1]
# 创建一个接受集合的函数，返回索引为 1 的元素
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [50]:
# 把多个参数传给 itemgetter，其构建的函数会返回提取的值构成的元组
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


In [51]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
              for name, cc, pop, (lat, long) in metro_data]
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [52]:
metro_areas[0].coord.lat

35.689722

In [53]:
# attrgetter 创建的函数根据名称提取对象属性
# 如果参数命中包含 . 则会深入嵌套对象
from operator import attrgetter

name_lat = attrgetter('name', 'coord.lat')
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [54]:
# methodcaller 创建的函数会在对象上调用参数指定的方法
from operator import methodcaller

s = 'The time has come'

upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [55]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-come'

methodcaller 还可以冻结某些参数，也就是部分应用（partial application）

部分应用是指，基于一个函数创建一个新的可调用对象，把原函数的某些参数固定。

functools.partial 这个高阶函数用于部分应用一个函数。使用这个函数可以把接受一个或多个参数的函数改编成需要回调的 API ，这样参数更少。

functools.partial 的第一个参数是一个可调用对象，后面跟着任意个要绑定的位置参数和关键字参数。
functools.partialmethod 函数的作用与 functools.partial 一样，不过是用于处理方法的。

functools 模块中的 lru_cache 函数能够做备忘（memorization），是一种自动优化措施，它会存储耗时的函数调用结果，避免重新计算。

In [56]:
from operator import mul
from functools import partial

triple = partial(mul, 3)
triple(7)

21

In [57]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [58]:
import unicodedata, functools

nfc = functools.partial(unicodedata.normalize, 'NFC')

s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

('café', 'café')

In [59]:
s1 == s2

False

In [60]:
nfc(s1) == nfc(s2)

True